In [0]:
!pip install GPyOpt


import torch
from torchvision import models, transforms, datasets
import time 
#Training code is inspired heavily from: https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
import os, sys
import copy
import GPyOpt
import numpy as np

Processing /root/.cache/pip/wheels/33/1d/87/dc02440831ba986b1547dd11a7dcd44e893b0527083066d869/GPyOpt-1.2.5-cp36-none-any.whl
  Using cached https://files.pythonhosted.org/packages/67/95/976598f98adbfa918a480cb2d643f93fb555ca5b6c5614f76b69678114c1/GPy-1.9.9.tar.gz
  Using cached https://files.pythonhosted.org/packages/d8/37/4abbeb78d30f20d3402887f46e6e9f3ef32034a9dea65d243654c82c8553/paramz-0.9.5.tar.gz
  Created wheel for GPy: filename=GPy-1.9.9-cp36-cp36m-linux_x86_64.whl size=2633995 sha256=10309cec54ee4ada66b3072f67d0150157a53782566f5ac73e8aeb0a06ccd14a
  Stored in directory: /root/.cache/pip/wheels/5d/36/66/2b58860c84c9f2b51615da66bfd6feeddbc4e04d887ff96dfa
  Created wheel for paramz: filename=paramz-0.9.5-cp36-none-any.whl size=102552 sha256=bb36adcfe6c857fc4097e5baba0ee3ccb4cc82089e3184dd3cdfa2abc721b8fe
  Stored in directory: /root/.cache/pip/wheels/c8/4a/0e/6e0dc85541825f991c431619e25b870d4b812c911214690cf8
Successfully built GPy paramz


In [0]:
def setup_dataset():

	data_transform = transforms.Compose([
			transforms.ToTensor(),
			transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
		])

	batch_size = 64

	print("Initializing Datasets and Dataloaders...")


	cifar10 = datasets.CIFAR10('local_data/', train = True,  transform = data_transform, download = True)
	cifar10test = datasets.CIFAR10('local_data/', train = False,  transform = data_transform, download = True)
	cifar10, _ = torch.utils.data.random_split(cifar10, [10000,len(cifar10)-10000])


	train_size = int(0.8 * len(cifar10))
	test_size = len(cifar10) - train_size
	cifar10train, cifar10val = torch.utils.data.random_split(cifar10, [train_size, test_size])
	image_datasets = {'train': cifar10train, 'val': cifar10val}
	
	dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}
	test_loader = torch.utils.data.DataLoader(cifar10test, batch_size=batch_size, shuffle=True, num_workers=4)
	return dataloaders_dict, test_loader



In [0]:
def train_model(hyperparameters, model, dataloaders, num_epochs=10):
	in_features, out_features = 25088, 10
	
	p = hyperparameters['p']
	hidden_units = hyperparameters['hidden_units']
	activation_func = hyperparameters['activation_func']

	
	model.classifier[0] = torch.nn.Linear(in_features, hidden_units)
	model.classifier[1] = activation_func
	model.classifier[2] = torch.nn.Dropout(p=p)
	model.classifier[3] = torch.nn.Linear(hidden_units, hidden_units)
	model.classifier[4] = activation_func
	model.classifier[5] = torch.nn.Dropout(p=p)
	model.classifier[6] = torch.nn.Linear(hidden_units, out_features)

	device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
	model = model.to(device)	

	criterion = torch.nn.CrossEntropyLoss()
	optimizer = torch.optim.Adam(model.parameters())


	for epoch in range(num_epochs):
		print('Epoch {}/{}'.format(epoch, num_epochs - 1))
		print('-' * 10)

		# Each epoch has a training and validation phase
		model.train()  # Set model to training mode

		running_loss = 0.0
		# Iterate over data.
		for inputs, labels in dataloaders['train']:
			inputs = inputs.to(device)
			labels = labels.to(device)

			# zero the parameter gradients
			optimizer.zero_grad()

			with torch.set_grad_enabled(True):
				# Get model outputs and calculate loss
				outputs = model(inputs)
				loss = criterion(outputs, labels)


				loss.backward()
				optimizer.step()

			
			running_loss += loss.item() * inputs.size(0)

		epoch_loss = running_loss / len(dataloaders['train'].dataset)
		

		print('Training Loss: {:.4f} '.format(epoch_loss))
		print()
		
	model.eval()
	corrects = 0
	for inputs, labels in dataloaders['val']:
		inputs = inputs.to(device)
		labels = labels.to(device)
		with torch.set_grad_enabled(False):
			# Get model outputs and calculate loss
			outputs = model(inputs)
			_, preds = torch.max(outputs, 1)
			corrects += torch.sum(preds == labels.data)

	val_acc = corrects.double() / len(dataloaders['val'].dataset)

	return model, float(val_acc)


In [0]:
DATALOADERS, test_loader = setup_dataset()

ACTIV = {0: torch.nn.Tanh(),
         1: torch.nn.ReLU(),
         2: torch.nn.ReLU6(),
         3: torch.nn.Sigmoid(),
        }


def objective_function(x):
  MODEL = models.vgg16(pretrained="JADAK!")

  for param in MODEL.features.parameters(): param.requires_grad = False

  x = x[0]
  hyperparameters = {
      'hidden_units': int(np.ceil(x[0]*4000))+int(1),
      'p': x[1],
      'activation_func': ACTIV[int(x[2])]
  }
  print(hyperparameters)
  trained_model, val_acc  = train_model(hyperparameters, MODEL, DATALOADERS)
  print(val_acc)
  return -val_acc


domain = [{'hidden_units'   : 'var_1', 'type': 'continuous', 'domain': (0 , 1)  },
          {'p'              : 'var_2', 'type': 'continuous',  'domain': (0 , 1)},
          {'activation_func': 'var_3', 'type': 'categorical','domain': tuple(np.arange(4))}]

opt = GPyOpt.methods.BayesianOptimization(f = objective_function,   # function to optimize
                                              domain = domain,         # box-constrains of the problem
                                              acquisition_type = 'EI',      # Select acquisition function MPI, EI, LCB
                                             )
opt.acquisition.exploration_weight=.1

opt.run_optimization(max_iter = 7)
x_best = opt.X[np.argmin(opt.Y)]


print("bedste: ", x_best)

In [0]:
latex_results = ""

for i, result in enumerate(opt.X[:]):
  latex_results += f"{i+1} & {int(np.ceil(result[0]*4000))} & {result[1]:.4} & {list(ACTIV.keys())[int(result[2])]} & {-float(opt.Y[i])} \\\\ \hline \n"

print(latex_results)

In [0]:


DATALOADERS, test_loader = setup_dataset()

ACTIV = {0: torch.nn.Tanh(),
         1: torch.nn.ReLU(),
         2: torch.nn.ReLU6(),
         3: torch.nn.Sigmoid(),
        }
MODEL = models.vgg16(pretrained="sooooo!")
for param in MODEL.features.parameters(): param.requires_grad = False
hidden = np.array([100,2000,4000]) ; drop = np.array([0.33, 0.66]) 
for i in range(3):
  for j in range(2):
    for k in range(2):
      if k == 1:
        activation_func = np.random.randint(0, high=4, size = 2)
      hyperparameters = {
          'hidden_units': int(hidden[i]),
          'p': drop[j],
          'activation_func': ACTIV[activation_func[k]]
      }
      print(hyperparameters)
      trained_model, val_acc  = train_model(hyperparameters, MODEL, DATALOADERS)
      print(val_acc)


In [0]:
DATALOADERS, test_loader = setup_dataset()

def hyperparam_tester(n_hidden, probability, activation_function):
    model = models.vgg16(pretrained="JADAK!")
    for param in model.features.parameters(): param.requires_grad = False
    model, val_acc  = train_model({'hidden_units': n_hidden, 'p': probability, 'activation_func': activation_function}, model, DATALOADERS)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    model.eval()
      
    corrects = 0
    for inputs, labels in test_loader:
      inputs = inputs.to(device)
      labels = labels.to(device)
      with torch.set_grad_enabled(False):
        # Get model outputs and calculate loss
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        corrects += torch.sum(preds == labels.data)

    test_acc = corrects.double() / len(test_loader.dataset)
    return test_acc


Initializing Datasets and Dataloaders...
Files already downloaded and verified
Files already downloaded and verified


In [0]:

print(hyperparam_tester(2540, 0.16, torch.nn.Tanh()))
print(hyperparam_tester(2403, 0.61, torch.nn.Sigmoid()))
print(hyperparam_tester(1150, 0.53, torch.nn.Sigmoid()))
print(hyperparam_tester(2000, 0.66, torch.nn.Sigmoid()))


Epoch 0/9
----------
Training Loss: 1.4856 

Epoch 1/9
----------
Training Loss: 1.1729 

Epoch 2/9
----------
Training Loss: 1.1129 

Epoch 3/9
----------
Training Loss: 1.0901 

Epoch 4/9
----------
Training Loss: 1.0358 

Epoch 5/9
----------
Training Loss: 1.0410 

Epoch 6/9
----------
Training Loss: 1.0321 

Epoch 7/9
----------
Training Loss: 1.0150 

Epoch 8/9
----------
Training Loss: 1.0362 

Epoch 9/9
----------
Training Loss: 1.0258 

tensor(0.6179, device='cuda:0', dtype=torch.float64)
Epoch 0/9
----------
Training Loss: 1.3831 

Epoch 1/9
----------
Training Loss: 1.1586 

Epoch 2/9
----------
Training Loss: 1.1290 

Epoch 3/9
----------
Training Loss: 1.0885 

Epoch 4/9
----------
Training Loss: 1.0713 

Epoch 5/9
----------
Training Loss: 1.0563 

Epoch 6/9
----------
Training Loss: 1.0446 

Epoch 7/9
----------
Training Loss: 1.0326 

Epoch 8/9
----------
Training Loss: 1.0113 

Epoch 9/9
----------
Training Loss: 1.0056 

tensor(0.6312, device='cuda:0', dtype=torch.flo